## Buoyancy driven flow 

Here we interpret buoyancy as having come from a thermal perturbation (not scaled)

**New concepts:** Different mesh types, initialising fields with numpy, a buoyancy _function_, 


**Stokes Flow**
\\[
\begin{aligned}
\tau_{ij,j} -  p_{,i} &= f_i && \text{ on } && \Omega \\
u_{i,i}&=0  && \text{ on } && \Omega \\
u_i &= \bar{u_i} && \text{ on } && \partial\Omega \\
\end{aligned}
\\]


- $\tau$ : deviatoric stress tensor
- $u$ : velocity
- $p$ : pressure
- $f$ : body force
- $\bar{u}$ : velocity boundary condition

![Results](../images/tempVel.png)

In [ ]:
import underworld as uw
from underworld import function as fn
import glucifer
import numpy as np

outputPath = 'output'

# create outputPath
import glob, json, os
# make a unique path
if os.path.exists(outputPath):
    outputPath += '_'+str(len(glob.glob(outputPath+str('*')))-1)

# build output dir string
if not outputPath.endswith('/'):
    outputPath += '/'
    
# make the output path
if not os.path.exists(outputPath):
    os.makedirs(outputPath)

In [ ]:
res = 32
boxHeight = 1.0
aspect_ratio = 1.0

In [ ]:
# A mesh to solve velocity and pressure

VPmesh = uw.mesh.FeMesh_Cartesian( elementType = ("Q1/dQ0"), 
                                   elementRes  = (int(res * aspect_ratio), res), 
                                   minCoord    = (0., 0.), 
                                   maxCoord    = (boxHeight*aspect_ratio, boxHeight))

# Another use case of functions... a different mesh
Tmesh = uw.mesh.FeMesh_Cartesian( elementType = ("Q1"), 
                                  elementRes  = (int(0.5*res * aspect_ratio), int(0.5*res )), 
                                  minCoord    = (0., 0.), 
                                  maxCoord    = (boxHeight*aspect_ratio, boxHeight))
# Tmesh = VPmesh

velocityField    = uw.mesh.MeshVariable( mesh=VPmesh,         nodeDofCount=2 ) # vector
pressureField    = uw.mesh.MeshVariable( mesh=VPmesh.subMesh, nodeDofCount=1 ) # scalar
temperatureField = uw.mesh.MeshVariable( mesh=Tmesh,          nodeDofCount=1 ) # scalar

velocityField.data[:,:] = 0.0
pressureField.data[:] = 0.0

# coordinate function
xcoordFn = fn.coord()[0]
ycoordFn = fn.coord()[1]

tempFn = fn.math.cos(xcoordFn * np.pi)*fn.math.sin(ycoordFn * np.pi)

# 'evaluate' now!
temperatureField.data[:] = tempFn.evaluate(temperatureField.mesh)

# The functions above are equivalent to the following
# temperatureField.data[:,0] = np.cos(Tmesh.data[:,0] * np.pi ) * np.sin(Tmesh.data[:,1] * np.pi)

# by using underworld functions: 
# 1) the operation is parallel safe. With numpy not the case!
# 2) performed using the data we give - could we do the operation on the VPmesh?
# 3) have a function descrption available for later on.

In [ ]:
# tempFn can be evaluate on raw coordinates as numpy array... automatic interpolation
inp = np.array( [
        (0.0,0.5), 
        (0.5,0.5) ])
tempFn.evaluate(inp)

In [ ]:
# Boundary conditions - free slip everywhere

vxWalls = VPmesh.specialSets["MinI_VertexSet"] + \
          VPmesh.specialSets["MaxI_VertexSet"] 

vyWalls = VPmesh.specialSets["MinJ_VertexSet"] + \
          VPmesh.specialSets["MaxJ_VertexSet"]

velocityBC = uw.conditions.DirichletCondition( variable = velocityField, 
                                               indexSetsPerDof = (vxWalls, vyWalls) )

In [ ]:
## Need a buoyancy function in the Stokes' solver

thermal_buoyancy = temperatureField * (0.0, 1.0)
 
stokesPIC = uw.systems.Stokes( velocityField  = velocityField, 
                               pressureField  = pressureField,
                               conditions     = [velocityBC,],
                               fn_viscosity   = 1.0, 
                               fn_bodyforce   = thermal_buoyancy )


solver = uw.systems.Solver( stokesPIC )

solver.solve()

In the above a standard mixed Galerkin FEM is applied via the **`Stokes Flow`** equation to generate a system of the form:

\\[
\begin{bmatrix}
 K & G \\
 G^T & 0
\end{bmatrix}
\begin{bmatrix}
  u\\
  p
\end{bmatrix}
=
\begin{bmatrix}
 f\\ 
 0
\end{bmatrix}.
\\]

where
- $K$ : the constitutive matrix, a function of `fn_viscosity`
- $u$ : velocity
- $p$ : pressure
- $f$ : body force, a function of `fn_bodyforce`
- $\bar{u}$ : velocity boundary condition, i.e. `velocityBC`

This system is solved using the **`Solver`** object.

In [ ]:
fig2 = glucifer.Figure()
fig2.append( glucifer.objects.VectorArrows( VPmesh, velocityField, arrowHead=0.35, scaling=3.0 ) )
fig2.append( glucifer.objects.Surface( VPmesh, temperatureField ) )
fig2.show()

## Things to try


- Does the temperature mesh have to be the same element type / resolution as the velocity mesh ?
- Experiment with a different temperature field to drive the fluid flow